<a href="https://colab.research.google.com/github/Mostofa-Najmus-Sakib/Recommender-System/blob/main/RecSys_CB_Disney_Mostofa.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Step by step details and context available at


*  https://analyticsindiamag.com/how-to-build-a-content-based-movie-recommendation-system-in-python/
*  https://medium.com/analytics-vidhya/content-based-recommender-systems-in-python-2b330e01eb80
 
 I mostly mixed and match to showcase how to build a simple content-based recommender from the ground up


Dataset: Diseny Plus shows from Kaggle
https://www.kaggle.com/unanimad/disney-plus-shows


In [1]:
import warnings
warnings.simplefilter('ignore')
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
cd /content/drive/My\ Drive/Boise \Class \Materials \(CS)/CS \537 \- \IR/Recommender_system/

/content/drive/My Drive/Boise Class Materials (CS)/CS 537 - IR/Recommender_system


In [3]:
#Load the full dataset
import pandas as pd
# from google.colab import files
# uploaded = files.upload()

df = pd.read_csv('disney_plus_shows.csv')

print(df)


       imdb_id  ... imdb_votes
0    tt0147800  ...    283,945
1    tt7019028  ...        124
2    tt0115433  ...     97,785
3    tt0324941  ...      7,434
4    tt0211181  ...     33,444
..         ...  ...        ...
987  tt3544734  ...      3,750
988  tt0186726  ...      6,378
989  tt0271271  ...      3,318
990  tt0379060  ...      2,339
991  tt2948356  ...    410,437

[992 rows x 19 columns]


In [4]:
#all the traits/data we could use to model items and therefore inform recommendation
print(df.columns) 

Index(['imdb_id', 'title', 'plot', 'type', 'rated', 'year', 'released_at',
       'added_at', 'runtime', 'genre', 'director', 'writer', 'actors',
       'language', 'country', 'awards', 'metascore', 'imdb_rating',
       'imdb_votes'],
      dtype='object')


In [5]:
#let us start with the basics: title and content description
df_content=df[['title', 'plot']]
df_content.head()

,title,plot
0,10 Things I Hate About You,"A pretty, popular teenager can't go out on a d..."
1,101 Dalmatian Street,This series follows the lives of Delilah and D...
2,101 Dalmatians,An evil high-fashion designer plots to steal D...
3,101 Dalmatians 2: Patch's London Adventure,"Being one of 101 takes its toll on Patch, who ..."
4,102 Dalmatians,Cruella DeVil gets out of prison and goes afte...


In [6]:
df.head()

,imdb_id,title,plot,type,rated,year,released_at,added_at,runtime,genre,director,writer,actors,language,country,awards,metascore,imdb_rating,imdb_votes
0,tt0147800,10 Things I Hate About You,"A pretty, popular teenager can't go out on a d...",movie,PG-13,1999,31 Mar 1999,"November 12, 2019",97 min,"Comedy, Drama, Romance",Gil Junger,"Karen McCullah, Kirsten Smith","Heath Ledger, Julia Stiles, Joseph Gordon-Levi...","English, French",USA,2 wins & 13 nominations.,70.0,7.3,"283,945"
1,tt7019028,101 Dalmatian Street,This series follows the lives of Delilah and D...,series,NaN,2018–,25 Mar 2019,"February 28, 2020",NaN,"Animation, Comedy, Family",NaN,NaN,"Josh Brener, Michaela Dietz, Bert Davis, Abiga...",English,"UK, USA, Canada",NaN,NaN,6.2,124
2,tt0115433,101 Dalmatians,An evil high-fashion designer plots to steal D...,movie,G,1996,27 Nov 1996,"November 12, 2019",103 min,"Adventure, Comedy, Crime, Family",Stephen Herek,"Dodie Smith (novel), John Hughes (screenplay)","Glenn Close, Jeff Daniels, Joely Richardson, J...","English, Spanish","USA, UK",Nominated for 1 Golden Globe. Another 3 wins &...,49.0,5.7,"97,785"
3,tt0324941,101 Dalmatians 2: Patch's London Adventure,"Being one of 101 takes its toll on Patch, who ...",movie,G,2002,21 Jan 2003,"November 12, 2019",74 min,"Animation, Adventure, Comedy, Family, Musical","Jim Kammerud, Brian Smith","Jim Kammerud (story), Dan Root (story), Garret...","Barry Bostwick, Jason Alexander, Martin Short,...",English,USA,5 wins & 10 nominations.,NaN,5.8,"7,434"
4,tt0211181,102 Dalmatians,Cruella DeVil gets out of prison and goes afte...,movie,G,2000,22 Nov 2000,"November 12, 2019",100 min,"Adventure, Comedy, Family",Kevin Lima,"Dodie Smith (novel), Kristen Buckley (story), ...","Glenn Close, Gérard Depardieu, Ioan Gruffudd, ...",English,"USA, UK",Nominated for 1 Oscar. Another 1 win & 7 nomin...,35.0,4.9,"33,444"


In [7]:
df.dropna(subset = ['plot', 'imdb_rating'],inplace=True)

In [8]:
#We are removing rows with missing values. In designing your own recommender, there are other alterantives than just remove, depending on the algorithmic strategy you choose for recommendations.
#Lack of full data representations for users and items if one of the main challenges recommenders have to face
print(df.isnull().sum())
#df.dropna(inplace=True)

imdb_id          0
title            0
plot             0
type             0
rated          122
year             0
released_at     14
added_at         0
runtime         38
genre            3
director       189
writer         133
actors          12
language        18
country         15
awards         304
metascore      567
imdb_rating      0
imdb_votes       0
dtype: int64


In [9]:
#What we have so far
df.head()

,imdb_id,title,plot,type,rated,year,released_at,added_at,runtime,genre,director,writer,actors,language,country,awards,metascore,imdb_rating,imdb_votes
0,tt0147800,10 Things I Hate About You,"A pretty, popular teenager can't go out on a d...",movie,PG-13,1999,31 Mar 1999,"November 12, 2019",97 min,"Comedy, Drama, Romance",Gil Junger,"Karen McCullah, Kirsten Smith","Heath Ledger, Julia Stiles, Joseph Gordon-Levi...","English, French",USA,2 wins & 13 nominations.,70.0,7.3,"283,945"
1,tt7019028,101 Dalmatian Street,This series follows the lives of Delilah and D...,series,NaN,2018–,25 Mar 2019,"February 28, 2020",NaN,"Animation, Comedy, Family",NaN,NaN,"Josh Brener, Michaela Dietz, Bert Davis, Abiga...",English,"UK, USA, Canada",NaN,NaN,6.2,124
2,tt0115433,101 Dalmatians,An evil high-fashion designer plots to steal D...,movie,G,1996,27 Nov 1996,"November 12, 2019",103 min,"Adventure, Comedy, Crime, Family",Stephen Herek,"Dodie Smith (novel), John Hughes (screenplay)","Glenn Close, Jeff Daniels, Joely Richardson, J...","English, Spanish","USA, UK",Nominated for 1 Golden Globe. Another 3 wins &...,49.0,5.7,"97,785"
3,tt0324941,101 Dalmatians 2: Patch's London Adventure,"Being one of 101 takes its toll on Patch, who ...",movie,G,2002,21 Jan 2003,"November 12, 2019",74 min,"Animation, Adventure, Comedy, Family, Musical","Jim Kammerud, Brian Smith","Jim Kammerud (story), Dan Root (story), Garret...","Barry Bostwick, Jason Alexander, Martin Short,...",English,USA,5 wins & 10 nominations.,NaN,5.8,"7,434"
4,tt0211181,102 Dalmatians,Cruella DeVil gets out of prison and goes afte...,movie,G,2000,22 Nov 2000,"November 12, 2019",100 min,"Adventure, Comedy, Family",Kevin Lima,"Dodie Smith (novel), Kristen Buckley (story), ...","Glenn Close, Gérard Depardieu, Ioan Gruffudd, ...",English,"USA, UK",Nominated for 1 Oscar. Another 1 win & 7 nomin...,35.0,4.9,"33,444"


In [10]:
#reorganize valid index now that some rows have been dropped
#inplace=True changes the original df
df.reset_index(drop=True,inplace=True)

In [11]:
#Modeling items based on their descriptions (plot), using TFIDF
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(analyzer='word', ngram_range=(1, 1), min_df=0, stop_words='english')
matrix = tf.fit_transform(df['plot'])

In [12]:
#Similarity across movies based on cosine similarity
from sklearn.metrics.pairwise import linear_kernel

cosine_similarities = linear_kernel(matrix,matrix)

In [13]:
#mapping title to df index
movie_title = df['title']
indices = pd.Series(df.index, index=df['title'])


In [14]:
#define a function for the recommendation; which will basically retrieve and rank the movies with the highest cosine similarity wrt a given one
def movie_recommend(title):

    idx = indices[title]

    sim_scores = list(enumerate(cosine_similarities[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
#Get the scores of the 15 most similar movies. Ignore the first movie.
    sim_scores = sim_scores[1:15]

    movie_indices = [i[0] for i in sim_scores]

    return movie_title.iloc[movie_indices]

In [15]:
#sample Top-5 recommendations 
movie_recommend("Frozen").head(15)

359            Lego Frozen Northern Lights
248                              Frozen II
18                     Alice in Wonderland
589                              Snow Dogs
487                                 On Ice
409    Captain America: The Winter Soldier
421           Marvel Super Hero Adventures
460                                  Moana
571                    Secret of the Wings
15                                 Aladdin
639                            Teen Spirit
14                                 Aladdin
735              The Prince and the Pauper
703                   The Legend of Mor'du
Name: title, dtype: object

In [16]:
#tweaking recommendation to resort "similar ones (content)" based on popularity of sorts (rating)
recommendations = movie_recommend("Frozen").head(15)
new_order = df[df.index.isin(recommendations.index)]
new_order.sort_values("imdb_rating", ascending=False)['title']

15                                 Aladdin
409    Captain America: The Winter Soldier
460                                  Moana
487                                 On Ice
18                     Alice in Wonderland
571                    Secret of the Wings
14                                 Aladdin
248                              Frozen II
703                   The Legend of Mor'du
735              The Prince and the Pauper
421           Marvel Super Hero Adventures
359            Lego Frozen Northern Lights
639                            Teen Spirit
589                              Snow Dogs
Name: title, dtype: object

In [17]:
#sample Top-5 recommendations 
movie_recommend('Frozen').head(5)

359    Lego Frozen Northern Lights
248                      Frozen II
18             Alice in Wonderland
589                      Snow Dogs
487                         On Ice
Name: title, dtype: object

In [18]:
#define a function for the recommendation; which will basically retrieve and rank based on Disney Score
def movie_recommend_DisneyScore(title):

    movie_indices = df.nlargest(5, ['metascore'])

    return movie_indices['title']

In [19]:
#sample Top-5 recommendations 
movie_recommend_DisneyScore('Frozen').head(5)

515                          Pinocchio
223                           Fantasia
541                        Ratatouille
46                Beauty and the Beast
590    Snow White and the Seven Dwarfs
Name: title, dtype: object

In [20]:
movie_recommend_DisneyScore('Snow Dogs').head(5)

515                          Pinocchio
223                           Fantasia
541                        Ratatouille
46                Beauty and the Beast
590    Snow White and the Seven Dwarfs
Name: title, dtype: object

In [21]:
#define a function for the recommendation; which will basically retrieve and rank based on Average Rating
def movie_recommend_rating(title):

    movie_indices = df.nlargest(25, ['imdb_rating'])

    return movie_indices['title']

In [22]:
#sample Top-5 recommendations 
movie_recommend_rating('Toy Story').head(15)

63                                              Bluey
694                            The Imagineering Story
145                                     Gravity Falls
199                              Dr. T, Lone Star Vet
493                                  One Strange Rock
384                       Lost Cities with Albert Lin
612    Star Wars: Episode V - The Empire Strikes Back
716                                   The Mandalorian
758                                      The Simpsons
605                Star Wars: Episode IV - A New Hope
695                            The Incredible Dr. Pol
480           Disney's Newsies: The Broadway Musical!
533                                   Primal Survivor
593                                          So Weird
707                                     The Lion King
Name: title, dtype: object

In [23]:
movie_recommend_rating('Minions').head(5)

63                      Bluey
694    The Imagineering Story
145             Gravity Falls
199      Dr. T, Lone Star Vet
493          One Strange Rock
Name: title, dtype: object

In [24]:
#to handle votes that do not originally have numer format
def convert_currency(val):
    """
    Convert the string number value to a float
     - Remove $
     - Remove commas
     - Convert to float type
    """
    new_val = val.replace(',','').replace('$', '')
    return float(new_val)

In [25]:
#to handle votes that do not originally have numer format
df['imdb_votes']=df['imdb_votes'].apply(convert_currency)

In [26]:
#define a function for the recommendation; which will basically retrieve and rank based on Popularity
def movie_recommend_popularity(title):
    movie_indices = df.nlargest(5, ['imdb_votes'])

    return movie_indices['title']

In [27]:
#sample Top-5 recommendations 
movie_recommend_popularity('Frozen').head(5)

417                                      The Avengers
605                Star Wars: Episode IV - A New Hope
612    Star Wars: Episode V - The Empire Strikes Back
38                                             Avatar
412                           Guardians of the Galaxy
Name: title, dtype: object

In [28]:
movie_recommend_popularity('Minions').head(5)

417                                      The Avengers
605                Star Wars: Episode IV - A New Hope
612    Star Wars: Episode V - The Empire Strikes Back
38                                             Avatar
412                           Guardians of the Galaxy
Name: title, dtype: object

In [29]:
df[['title', 'imdb_votes','imdb_rating']].head()

,title,imdb_votes,imdb_rating
0,10 Things I Hate About You,283945.0,7.3
1,101 Dalmatian Street,124.0,6.2
2,101 Dalmatians,97785.0,5.7
3,101 Dalmatians 2: Patch's London Adventure,7434.0,5.8
4,102 Dalmatians,33444.0,4.9


In [30]:
columnVotes = df["imdb_votes"]
max_votes = columnVotes.max()
columnRating = df["imdb_rating"]
max_rating = columnRating.max()
df["CombinedIMDB"] = ((df["imdb_votes"] /1225316.0) + (df["imdb_rating"]/max_rating))
df.head()

,imdb_id,title,plot,type,rated,year,released_at,added_at,runtime,genre,director,writer,actors,language,country,awards,metascore,imdb_rating,imdb_votes,CombinedIMDB
0,tt0147800,10 Things I Hate About You,"A pretty, popular teenager can't go out on a d...",movie,PG-13,1999,31 Mar 1999,"November 12, 2019",97 min,"Comedy, Drama, Romance",Gil Junger,"Karen McCullah, Kirsten Smith","Heath Ledger, Julia Stiles, Joseph Gordon-Levi...","English, French",USA,2 wins & 13 nominations.,70.0,7.3,283945.0,0.984309
1,tt7019028,101 Dalmatian Street,This series follows the lives of Delilah and D...,series,NaN,2018–,25 Mar 2019,"February 28, 2020",NaN,"Animation, Comedy, Family",NaN,NaN,"Josh Brener, Michaela Dietz, Bert Davis, Abiga...",English,"UK, USA, Canada",NaN,NaN,6.2,124.0,0.639276
2,tt0115433,101 Dalmatians,An evil high-fashion designer plots to steal D...,movie,G,1996,27 Nov 1996,"November 12, 2019",103 min,"Adventure, Comedy, Crime, Family",Stephen Herek,"Dodie Smith (novel), John Hughes (screenplay)","Glenn Close, Jeff Daniels, Joely Richardson, J...","English, Spanish","USA, UK",Nominated for 1 Golden Globe. Another 3 wins &...,49.0,5.7,97785.0,0.667433
3,tt0324941,101 Dalmatians 2: Patch's London Adventure,"Being one of 101 takes its toll on Patch, who ...",movie,G,2002,21 Jan 2003,"November 12, 2019",74 min,"Animation, Adventure, Comedy, Family, Musical","Jim Kammerud, Brian Smith","Jim Kammerud (story), Dan Root (story), Garret...","Barry Bostwick, Jason Alexander, Martin Short,...",English,USA,5 wins & 10 nominations.,NaN,5.8,7434.0,0.604005
4,tt0211181,102 Dalmatians,Cruella DeVil gets out of prison and goes afte...,movie,G,2000,22 Nov 2000,"November 12, 2019",100 min,"Adventure, Comedy, Family",Kevin Lima,"Dodie Smith (novel), Kristen Buckley (story), ...","Glenn Close, Gérard Depardieu, Ioan Gruffudd, ...",English,"USA, UK",Nominated for 1 Oscar. Another 1 win & 7 nomin...,35.0,4.9,33444.0,0.532449


In [31]:
#define a function for the recommendation; which will basically retrieve and rank based on combined info
def movie_recommend_popularity(title):
    movie_indices = df.nlargest(5, ['CombinedIMDB'])

    return movie_indices['title']

In [32]:
#sample Top-5 recommendations 
movie_recommend_popularity('Toy Story').head(5)

605                Star Wars: Episode IV - A New Hope
417                                      The Avengers
612    Star Wars: Episode V - The Empire Strikes Back
38                                             Avatar
412                           Guardians of the Galaxy
Name: title, dtype: object